In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
file_path = 'ANN-2.xlsx'

In [27]:
data = pd.read_excel(file_path)

In [28]:
data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,Sliding velocity,NaN,Normal load,Filler content,Sliding distance,Specific wear rate
4,NaN,NaN,NaN,NaN,NaN,A (m/s),B (N),C (%),D (m),Ws (mm3/N-km)


In [29]:
data = data.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'])

In [30]:
data.columns = data.iloc[3]
data = data.drop(index=[0, 1, 2, 3])


In [31]:
data = data.dropna(axis=1, how='all')

In [32]:
data.reset_index(drop=True, inplace=True)
data = data.apply(pd.to_numeric, errors='coerce')

In [33]:
data.head()

3,NaN,Normal load,Filler content,Sliding distance,Specific wear rate
0,NaN,NaN,NaN,NaN,NaN
1,0.5,10.0,5.0,500.0,0.132890
2,0.5,10.0,10.0,1000.0,0.113821
3,0.5,10.0,15.0,1500.0,0.074015
4,0.5,20.0,5.0,1000.0,0.161960


In [34]:
data.isnull().sum()

3
NaN                   1
Normal load           1
Filler content        1
Sliding distance      1
Specific wear rate    1
dtype: int64

In [35]:
data.dropna(inplace=True)

In [36]:
data.shape

(27, 5)

In [37]:
data.rename(columns={data.columns[0]: 'A (m/s)'}, inplace=True)


In [38]:
data.head()

3,A (m/s),Normal load,Filler content,Sliding distance,Specific wear rate
1,0.5,10.0,5.0,500.0,0.132890
2,0.5,10.0,10.0,1000.0,0.113821
3,0.5,10.0,15.0,1500.0,0.074015
4,0.5,20.0,5.0,1000.0,0.161960
5,0.5,20.0,10.0,1500.0,0.132791


In [39]:
x=data.drop("Specific wear rate",axis=1)
y=data["Specific wear rate"]

In [40]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [65]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def pushpa(model):
    # Fit the model and make predictions
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    # Calculate and print errors
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Absolute Error (MAE): {mae}")
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error (MSE): {mse}")
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error (RMSE): {rmse}")

    # Get input values from the user for the features
    list_1 = []
    for a in x_train.columns:
        while True:
            try:
                value = input(f"Enter the value of {a}: ")
                if value.strip() == "":
                    raise ValueError("Input cannot be empty")
                list_1.append(float(value))
                break
            except ValueError as e:
                print(f"Invalid input: {e}. Please enter a valid number for {a}.")

    # Predict based on user input
    list_input = np.array(list_1).reshape(1, -1)
    result = model.predict(list_input)
    print(f"The predicted value is: {result[0]}")


In [66]:
from sklearn.linear_model import LinearRegression
pushpa(LinearRegression())

Mean Absolute Error (MAE): 12.585774458167847
Mean Squared Error (MSE): 177.65795481584246
Root Mean Squared Error (RMSE): 13.32883921486948
The predicted value is: -9.26108479602923


C:\Users\GUNATEJA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [42]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [43]:
x_t=data.drop("Specific wear rate",axis=1)
Y=data["Specific wear rate"]

In [44]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(x_t)

In [45]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

In [46]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

C:\Users\GUNATEJA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [48]:
history = model.fit(X_train, Y_train, epochs=100, batch_size=10, validation_split=0.2, verbose=1)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - loss: 0.0938 - mae: 0.2980 - val_loss: 0.0549 - val_mae: 0.2261
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0537 - mae: 0.2233 - val_loss: 0.0296 - val_mae: 0.1604
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0312 - mae: 0.1617 - val_loss: 0.0139 - val_mae: 0.0995
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0149 - mae: 0.1060 - val_loss: 0.0063 - val_mae: 0.0528
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0079 - mae: 0.0782 - val_loss: 0.0044 - val_mae: 0.0515
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0061 - mae: 0.0636 - val_loss: 0.0053 - val_mae: 0.0658
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0068 - mae: 0.0627 - val_loss: 0.0068 - val_mae: 0.0765
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0074 - mae: 0.0639 - val_loss: 0.0077 - val_mae: 0.0806
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0086 - mae:

In [49]:
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on test data: {mae}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0036 - mae: 0.0515
Mean Absolute Error on test data: 0.05153775215148926


In [50]:
import numpy as np
list_1 = []

for a in x_train.columns:
    while True:
        try:
            value = input(f"Enter the value for {a}: ")
            if value.strip() == "":
                raise ValueError("Input cannot be empty.")
            list_1.append(float(value))
            break
        except ValueError as e:
            print(f"Invalid input: {e}. Please enter a valid number for {a}.")
input_array = np.array(list_1).reshape(1, -1)
input_scaled = scaler.transform(input_array)
result = model.predict(input_scaled)
print(f"The predicted value is: {result[0][0]} Ws (mm³/N-km)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
The predicted value is: 0.1316930651664734 Ws (mm³/N-km)


C:\Users\GUNATEJA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
